# 2D FEM Setup

## Framework

The framework is designed to accompany these features:
- MPI-based domain decomposition
- parallelized, iterative solver that connects seamlessly
- elastic deformation via fft-based convolution

<div style="text-align: center;">
  <img src="./figures/2D_FEM_Framework.png" 
       alt="2D FEM framework" 
       width="1000">
</div>

## FEM Grid and Elements

### Basis Element

We use triangular basis elements with three quadrature points.

```python
# Barycentric coordinates for 3-point Gauss quadrature on triangle
# each row is one quad point, columns are barycentric coords for nodes 0,1,2
BARY_COORDS = np.array([
    [2 / 3, 1 / 6, 1 / 6],
    [1 / 6, 2 / 3, 1 / 6],
    [1 / 6, 1 / 6, 2 / 3],
])

# Quadrature weights; sum up to 0.5 for one triangle
WEIGHTS = np.array([1 / 6, 1 / 6, 1 / 6])
```

Note that each square spanned by four grid points hold two elements.

<div style="text-align: center;">
  <img src="./figures/2D_FEM_point_names.png"
       alt="2D FEM point names"
       width="200">
</div>

Effectively, we get a 7-point stencil. The test function of one point interacts with trial functions of 6 neighbouring points and itself.

Note that point (0,0) does not interact with (1,1) and (-1,-1) due to the selected orientation of the triangles.

<div style="text-align: center;">
  <img src="./figures/2D_FEM_Stencil.png" 
       alt="2D FEM Stencil" 
       width="300">
</div>

### Grid Layout

The 2D grid uses column-major ordering with shape `(Nx_padded, Ny_padded)`.

- Axis 0 = X (West → East)
- Axis 1 = Y (South → North)
- Access: `field[x, y]`

In FEM we need to go through all points of the domain and capture all dependencies of the point w.r.t. neighbouring points.

To do this in a systematic way, we iterate through the squares spanned by the points. Each square has the similar structure, which allows vectorization.

As a first step, we need to know how to map from `square i,j` $\rightarrow$ point indices of `(bl, br, tl, tr)`.
Then we can capture interactions within the left `(bl, br, tl)` and right `(br, tl, tr)` triangle.

The figure below illustrates the mapping. Note that in the actual implementation, we use pre-computed mappings that are more complex but work on the similar idea.

<div style="text-align: center;">
  <img src="./figures/2D_FEM_pixels_points.png" 
       alt="2D FEM pixels and points" 
       width="800">
</div>

## Domain Decomposition

Domain decomposition is implemented using the `CartesianDecomposition` feature of [muGrid](https://github.com/muSpectre/muGrid). The following sections shortly address the domain decomposition functionality of muGrid, how boundary conditions are treated, and how grid indexing is implemented in order to correctly assemble a global equation system from the local parts of the decomposed domain.

### muGrid

muGrid offers a <code style="color:#c586c0">CartesianDecomposition</code> object that requires an MPI communicator, grid size, number of ghost cells in all directions, and shape of subdivision.

```python
decomp = CartesianDecomposition(...)
```
<br>

In our case, the subdivision is set to

```python
nb_subdivisions = (1, comm.size)
```
<br>

meaning that each process owns a slice with 
- full x-length and 
- part of the y-range.

<br>

Now fields can be instanciated "under" the decomposition object. When using these fields, each process will only see its own slice of the field.

```python
field = decomp.real_field('solution)
```
<br>

Interface to fields:
- `field.p` accesses the inner field
- `field.pg` accesses the padded field with ghost cells
- Writing needs to be done using the slicing operator `field.p[:] = arr`

<br>

Ghost cells either represent adjacent domains or periodic wrap-around. After the inner field is updated, ghost cells need to be separately updated using:

```python
decomp.communicate_ghosts(field)
```

### Boundary Conditions

To take advantage of the muGrid-based framework, we use ghost cells as a means to impose boundary conditions. Thereby, we do not require injecting boundary conditions into the equation system during assembly.

- <code style="color:#4CAF50">Dirichlet BC</code>: Boundary values are directly written to ghost cells. This is slightly different to the current explicit solver, where we demand the BC value to be satisfied **between** the last inner node and the ghost cell
- <code style="color:#4CAF50">Neumann BC</code>: The ghost cell value is calculated using information from only the direct neighbour

### Local vs. Global Indexing

The challenge of the parallelized approach is to correctly join the local equations systems into a global system. Therefore, each process has to know where its slice is positioned in the global domain. The basis of this translation is given by index masks.

Assume we have a `(3,4)` domain and run on two processes. Each process then has a `(3,2)` sub-domain (and a `(5,4)` padded domain that includes ghost cells).

<code style="color:#c586c0">Global Domain Indexing</code>:

$$
\begin{pmatrix}
\begin{array}{cc:cc}
0 & 3 & 6 & 9 \\
1 & 4 & 7 & 10 \\
2 & 5 & 8 & 11
\end{array}
\end{pmatrix}
$$

<code style="color:#4CAF50">Process 1</code>:

$$
\mathrm{Local \, indices:} \quad
\begin{pmatrix}
0 & 3 \\
1 & 4 \\
2 & 5 \\
\end{pmatrix}
\quad \quad
\mathrm{Global \, indices:} \quad
\begin{pmatrix}
0 & 3 \\
1 & 4 \\
2 & 5 \\
\end{pmatrix}
$$

<code style="color:#4CAF50">Process 2</code>:

$$
\mathrm{Local \, indices:} \quad
\begin{pmatrix}
0 & 3 \\
1 & 4 \\
2 & 5 \\
\end{pmatrix}
\quad \quad
\mathrm{Global \, indices:} \quad
\begin{pmatrix}
6 & 9 \\
7 & 10 \\
8 & 11 \\
\end{pmatrix}
$$

Note that it gets a little more complex since e.g. on process 1, the points 3,4,5 interact with (global) points 6,7,8 of process 2 (and if periodic in y, the points 0,1,2 interact with 9,10,11). Therefore, each process also has to know global indices of its ghost cells. See example for process 2 below. 

The `-1` stands for Dirichlet boundary condition. In case of Neumann boundary condition, the index of the neighbouring point has to be "forwarded" to the ghost point to correctly capture its influence. Since Dirichlet/Neumann is variable-specific (for example Dirichlet for density but Neumann for flux) these index-masks need to be determined and used variable-specific.

For simplicity, we assume Dirichlet BCs on all sides. Note that regarding the indexing, inner nodes are filled first, followed by ghost cells that are not Dirichlet-type boundaries. This has to do with the fact that on each rank, only the residuals of the inner points are evaluated.

<code style="color:#4CAF50">Process 2</code>:

$$
\mathrm{Local \, padded \, indices:} \quad
\begin{pmatrix}
-1 & -1 & -1 & -1 \\
6 &  0 &  3 & -1 \\
7 &  1 &  4 & -1 \\
8 &  2 &  5 & -1 \\
-1 & -1 & -1 & -1
\end{pmatrix}
\quad\quad
\mathrm{Global \, padded \, indices:} \quad
\begin{pmatrix}
-1 & -1 & -1 & -1 \\
3 &  6 &  9 & -1 \\
4 &  7 &  10 & -1 \\
5 &  8 &  11 & -1 \\
-1 & -1 & -1 & -1
\end{pmatrix}
$$

## Assembly

The equation system yields a sparse tangential matrix. Note that storing the full matrix is not performant and quickly reaches memory bottleneck.

> Example: <br> 128x128 grid $\rightarrow$ 16.384 points, with four equations per point $\rightarrow$ 65.536 equations $\rightarrow$ 65.536^2 = 4.3x10^9 floats in the tangential matrix $\approx$ 32 GB memory

However, the number of non-zeros (nnz) is - with our stencil - at maximum 7x4 = 28 per equation (residual is influenced by this number of variables). For performant assembly and for consistency with `petsc`, we use `COO` (Coordinate List) for efficiently capturing only nnz values.

Assume you want to represent this matrix in `COO` format:
$$
A = 
\begin{pmatrix}
10 & 0 & 0 \\
0 & 0 & 20 \\
0 & 30 & 0
\end{pmatrix}
$$

```python
row = [0, 1, 2]
col = [0, 2, 1]
values = [10, 20, 30]
```

---

**Block vs. Point-interleaved ordering**

This is an important concept in order to understand the implementation approach.

During assembly, we iterate through the individual terms the residuals consist of. And for each term, we iterate through dependent variables.

> Example: <br> The mass conservation residual contains the $- \frac{\partial j_x}{\partial x}$ term. There is only one dependent variable: $j_x$.

The point is that each step consists of determining the change of `residual X` due to `variable Y`. So a block is represented by an `X-Y` combination.

And since we evaluate $\sim$ 40 of these blocks in each assembly, it is performant to let each block be represented by a contiguous piece on the `COO` list.

---

The second part of the story is the relation between the local and the global equation system.

We want the equations of the local equation system (tangential matrix of the process-specific subdomain) to be placed on a contiguos piece of the global equation system.

For this, we need point-interleaved ordering of the system.

Here is an illustration assuming a small `(2,2)` system with two residuals/equations `r1`, `r2` and two variables/fields `v1`, `v2`. It is decomposed on two processes with `(2,1)` subdomains.


<code style="color:#4CAF50">Process 1</code> owns the points `[0, 1]` and the residuals and variables that live on them.

These are the residuals (rows) $\color{ForestGreen}{0\text{--}r_1, 0\text{--}r_2}, 1\text{--}r_1, 1\text{--}r_2$ and variables (column) $\color{ForestGreen}{0\text{--}v_1, 0\text{--}v_2}, 1\text{--}v_1, 1\text{--}v_2$. Convention here: #point-var/residual

If the global matrix used **block format**, the content of process 1 would be scattered across it:

$$
\begin{array}{c|cccccccc}
 &
 0\text{--}v_1 & 1\text{--}v_1 & 2\text{--}v_1 & 3\text{--}v_1 &
 0\text{--}v_2 & 1\text{--}v_2 & 2\text{--}v_2 & 3\text{--}v_2
\\ \hline

0\text{--}r_1 &
\color{green}{*} & \color{green}{*} & * & * &
\color{green}{*} & \color{green}{*} & * & *
\\

1\text{--}r_1 &
\color{green}{*} & \color{green}{*} & * & * &
\color{green}{*} & \color{green}{*} & * & *
\\

2\text{--}r_1 &
* & * & * & * & * & * & * & *
\\

3\text{--}r_1 &
* & * & * & * & * & * & * & *
\\

0\text{--}r_2 &
\color{green}{*} & \color{green}{*} & * & * &
\color{green}{*} & \color{green}{*} & * & *
\\

1\text{--}r_2 &
\color{green}{*} & \color{green}{*} & * & * &
\color{green}{*} & \color{green}{*} & * & *
\\

2\text{--}r_2 &
* & * & * & * & * & * & * & *
\\

3\text{--}r_2 &
* & * & * & * & * & * & * & *
\end{array}
$$


But using **point-interleaved ordering**, process 1 is represented by a contiguous block in the global matrix:

$$
\begin{array}{c|cccccccc}
 &
 0\text{--}v_1 & 0\text{--}v_2 & 1\text{--}v_1 & 1\text{--}v_2 &
 2\text{--}v_1 & 2\text{--}v_2 & 3\text{--}v_1 & 3\text{--}v_2
\\ \hline

0\text{--}r_1 &
\color{green}{*} & \color{green}{*} & \color{green}{*} & \color{green}{*} & * & * & * & *
\\

0\text{--}r_2 &
\color{green}{*} & \color{green}{*} & \color{green}{*} & \color{green}{*} & * & * & * & *
\\

1\text{--}r_1 &
\color{green}{*} & \color{green}{*} & \color{green}{*} & \color{green}{*} & * & * & * & *
\\

1\text{--}r_2 &
\color{green}{*} & \color{green}{*} & \color{green}{*} & \color{green}{*} & * & * & * & *
\\

2\text{--}r_1 &
* & * & * & * & * & * & * & *
\\

2\text{--}r_2 &
* & * & * & * & * & * & * & *
\\

3\text{--}r_1 &
* & * & * & * & * & * & * & *
\\

3\text{--}r_2 &
* & * & * & * & * & * & * & *
\end{array}
$$



> Therefore, a translation is necessary from block to point-interleaved ordering.

This is implemented by using two `COO` lists. The `values` array is the same for both.

- `local_rows`, `local_cols` - used during block-wise iteration for assembly (one block is a contiguous piece on the `values` array)
- `global_rows`, `global_cols` - translation to global equation system

---

**Overview of important containers**

All index mappings are computed once during setup and are stored in these containers:

```
FEMAssemblyLayout        top-level container
├── MatrixCOOPattern     COO local and global indices
├── RHSPattern           RHS vector indices
├── ScalingInfo          diagonal scaling factors
└── jacobian_terms       holds term/variable specific indices and precomputed values
```

Note that similar containers exist for the rhs, but details are left out here.

<code style="color:#c586c0">TermKey</code> is a combination of term and dependent variable.

<br>

**MatrixCOOPattern**

Stores the sparse matrix structure in COO (Coordinate) format.

The `MatrixCOOPattern` lists are determined first, and `JacobianTermMap`s are then created based on them.

| Attribute | Type | Description |
|-----------|------|-------------|
| `local_rows` | IntArray | Local row indices for each COO entry |
| `local_cols` | IntArray | Local column indices for each COO entry |
| `global_rows` | IntArray | Global row indices (for PETSc/MPI) |
| `global_cols` | IntArray | Global column indices (for PETSc/MPI) |
| `res_block_idx` | Int8Array | Residual block index (0=mass, 1=mom_x, ...) |
| `var_block_idx` | Int8Array | Variable block index (0=ρ, 1=jx, ...) |

The block indices enable per-variable scaling without recomputing the COO structure.

<br>

**JacobianTermMap** - for each <code style="color:#c586c0">TermKey</code>

Maps contributions from one block (term-variable) to matrix entries.

| Attribute | Type | Description |
|-----------|------|-------------|
| `coo_idx` | IntArray | Which COO entries receive contributions |
| `weights` | NDArray | precomputed integration weights |
| `sq_x`, `sq_y` | IntArray | Square indices for field value lookup |

<br>

**FEMAssemblyLayout**

Top-level container that holds everything.

| Attribute | Type | Description |
|-----------|------|-------------|
| `matrix_coo` | MatrixCOOPattern | Sparse matrix structure |
| `rhs` | RHSPattern | RHS vector structure |
| `jacobian_terms` | dict | Maps <code style="color:#c586c0">TermKey</code> → `JacobianTermMap` |

The `get_petsc_info()` method extracts PETSc-specific assembly info, and `build_scaling()` constructs diagonal scaling factors.

---

### Diagonal Scaling

Variables in the system span different magnitudes (e.g., ρ~10³ kg/m³, E~10⁸ J/m³), leading to an ill-conditioned Jacobian. Diagonal scaling improves conditioning without modifying the physics.

**Transformation**

The linear system $J \cdot \delta q = -R$ is transformed to:

$$
J^* \cdot \delta q^* = -R^*
$$

where:
- $D_q = \text{diag}(q_\text{scale})$ — variable scaling (characteristic magnitudes)
- $D_R = \text{diag}(R_\text{scale})$ — residual scaling (same as $D_q$ by convention)
- $J^* = D_R^{-1} \cdot J \cdot D_q$
- $R^* = D_R^{-1} \cdot R$
- $\delta q = D_q \cdot \delta q^*$

<br>

**ScalingInfo**

| Attribute | Type | Description |
|-----------|------|-------------|
| `coo_scale` | NDArray | Per-entry scaling for COO values: $1/(R_\text{scale} \cdot q_\text{scale})$ |
| `rhs_scale` | NDArray | Per-DOF residual scaling |
| `sol_scale` | NDArray | Per-DOF solution unscaling |
| `char_scales` | dict | Characteristic scales: `{'rho': ..., 'jx': ..., 'jy': ..., 'E': ...}` |

Characteristic scales are derived from the problem specification:
- $\rho_\text{ref}$ from `rho0`
- $j_\text{ref} = \rho_\text{ref} \cdot \max(|U|, |V|)$
- $E_\text{ref} = \rho_\text{ref} \cdot c_v \cdot T_\text{ref}$ (if energy equation active)

## Solver

For parallelized solving of the equation system, we use `PETSc` since it provides efficient, scalable solvers and preconditioners for large sparse systems on parallel architectures.

The following container holds all information that is exposed to `PETScSystem`, which wraps the `PETSc` functionality.

**PETScAssemblyInfo**

| Attribute | Type | Description |
|-----------|------|-------------|
| `local_size` | Int | Number of equations in the local system |
| `global_size` | Int | Number of equations in the global system |
| `mat_global_rows` | IntArray | Global `COO` indices |
| `mat_global_cols` | IntArray | Global `COO` indices |
| `rhs_global_rows` | IntArray | RHS indices |


Important lines for translation of local assembly to global system within `PETScSystem` are:

```python
class PETScSystem:

    def _create_petsc_objects(self):
        self.mat = PETSc.Mat().create(self.comm)  # create matrix
        self.mat.setSizes([(local_size, global_size), (local_size, global_size)])  # set local and global sizes (decomposition information)
        self.mat.setPreallocationCOO(info.mat_global_rows, info.mat_global_cols)  # provide COO information

    def assemble(self, coo_values: NDArray):
        self.mat.setValuesCOO(coo_values, PETSc.InsertMode.INSERT_VALUES) # set values using COO information
```

---

**`PETSc` alternative for serial execution**

When PETSc is unavailable, `ScipySystem` provides a serial fallback using SciPy's sparse direct solver (`splu`). Note that this fallback is automatically activated in serial execution when `PETSc` is not available. When running in parallel without `PETSc`, an error is raised.

```python
class ScipySystem:
    def assemble(self, coo_values, R_local):
        self.mat = csr_matrix((coo_values, (rows, cols)), shape=(n, n))
        self.rhs = -R_local

    def solve(self, ...):
        return splu(self.mat).solve(self.rhs)
```

## Elastic Deformation

Elastic deformation is computed using the Green's function based approach, where the required convolution operation can be efficiently conducted in Fourier space see [Elastic Deformation Tutorial](07_elastic_deformation.ipynb).

We use the <code style="color:#c586c0">muGrid.FFTEngine</code> to handle parallelized FFT operations. The `FFTEngine` acts as a wrapper similar to `CartesianDecomposition` and offers `fft` and `ifft` operations. 

Note that we set the subdivision of `CartesianDecomposition` to match the non-customizable subdivision of `FFTEngine`.
In parallelized runs, we can therefore directly use the local pressure fields in `CartesianDecomposition` as input for fft operations.

Unfortunately however, this is only true for fully periodic domains. For semi-periodic or fully non-periodic domains, the fft approach for elastic deformation requires a decoupling trick, which in turn requires padding the domain from $N_x, N_y \rightarrow 2 N_x, 2 N_y$. In this case, the local data needs to be massively redistributed (compare figure below).



<div style="text-align: center;">
  <img src="./figures/FFTDomainTranslation.png" 
       alt="2D FEM framework" 
       width=500">
</div>

This redistribution is handled by the <code style="color:#c586c0">FFTDomainDecomposition</code> wrapper.
Here is a small overview of the most important lines. 

Note that when simulating a fully periodic domain, the exchange plan is basically a 1:1 copy of local fields.

```python
class FFTDomainTranslation:
    def __init__(self, ...):
        self._compute_fft_grid_size()  # compute FFT grid size based on periodicity
        self.fft_engine = muGrid.FFTEngine([self.Nx_fft, self.Ny_fft], comm)  # create FFTEngine with computed size
        self._build_exchange_plan()  # core: build exchange plan for MPI redistribution

    def embed(self, src: np.ndarray, dst: np.ndarray):
        # CartesianDecomposition -> FFTDeomposition

    def extract(self, src: np.ndarray, dst: np.ndarray):
        # FFTDecomposition -> CartesianDecomposition
```

When computing displacement from elastic deformation, the wrapper functions of <code style="color:#c586c0">FFTDomainDecomposition</code> are used as follows:

```python
class ElasticDeformation:
    def get_deformation(self, p: NDArray)
    
        # redistribute pressure field on padded domain
        self.fft_translation.embed(p, p_padded_domain)

        # calculate force and get displacement via fft (on padded domain)
        force_padded_domain = self.area_per_cell * p_padded_domain
        disp_padded_domain = - self.ElDef.evaluate_disp(force_padded_domain)

        # collect displacement field from padded domain
        self.fft_translation.extract(disp_padded_domain, displacement)
    
        return displacement  # displacement on local CartesianDecomposition field
```

## Implementation Overview

| File | Description |
|------|-------------|
| `solver_fem_2d.py` | Main 2D FEM solver: Newton iteration, assembly dispatch, time-stepping |
| `fem_2d/elements.py` | <code style="color:#c586c0">TriangleQuadrature</code> - shape functions, derivatives, interpolation operators |
| `fem_2d/terms.py` | <code style="color:#c586c0">NonLinearTerm</code>, `get_active_terms()`, residual terms (`R11x`, `R11y`, ...) |
| `fem_2d/assembly_layout.py` | <code style="color:#c586c0">FEMAssemblyLayout</code>, <code style="color:#c586c0">MatrixCOOPattern</code>, <code style="color:#c586c0">RHSPattern</code>, <code style="color:#c586c0">ScalingInfo</code>, <code style="color:#c586c0">JacobianTermMap</code>, <br> <code style="color:#c586c0">PETScAssemblyInfo</code> - precomputed indexing, local-global translation, scaling, and weightings |
| `fem_2d/grid_index.py` | <code style="color:#c586c0">GridIndexManager</code> - local/global index masks, BC handling, stencil connectivity |
| `fem_2d/quad_fields.py` | <code style="color:#c586c0">QuadFieldManager</code> - field storage, nodal→quadrature interpolation |
| `fem_2d/petsc_system.py` | <code style="color:#c586c0">PETScSystem</code> - distributed matrix assembly and KSP linear solve (parallel/serial) |
| `fem_2d/scipy_system.py` | <code style="color:#c586c0">ScipySystem</code> - serial sparse solver fallback when PETSc unavailable |

# 2D Equations

Overview of 2D equations solved by the FEM framework.

## Mass Conservation

Mass conservation connects density $\rho$ and mass fluxes $j_x$, $j_y$:

$$
\begin{aligned}
0 &= 
-\,\frac{\overline{\rho} - \overline{\rho}_{\mathrm{prev}}}{\Delta t} \\[4pt]
&\quad
-\,\frac{\partial \overline{j_x}}{\partial x}
- \frac{1}{h}\,\frac{\partial h}{\partial x}\,\overline{j_x} \\[4pt]
&\quad
-\,\frac{\partial \overline{j_y}}{\partial y}
- \frac{1}{h}\,\frac{\partial h}{\partial y}\,\overline{j_y} \\[4pt]
&\quad
-\,\overline{\rho}\,\frac{1}{h}\,\frac{\partial h}{\partial t} \\[4pt]
&\quad
-\,\alpha_p \left( \frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial y^2} \right)
\end{aligned}
$$

- **R1T**: Time derivative
- **R11x**: Flux divergence in x
- **R11Sx**: Flux divergence height source in x
- **R11y**: Flux divergence in y
- **R11Sy**: Flux divergence height source in y
- **R12**: Height change over time (not implemented)
- **R1Stabx**, **R1Staby**: Pressure stabilization (Brezzi-Pitkäranta style for equal-order elements)

## Momentum x

$$
\begin{aligned}
0 &= 
- \frac{\overline{j_x} - \overline{j_x}_{\mathrm{prev}}}{\Delta t} \\[2pt]
&\quad
- \frac{\partial p}{\partial x} \\[2pt]
&\quad
- \frac{\partial}{\partial x}\!\left( \frac{\overline{j_x}^2}{\overline{\rho}} \right)
- \frac{1}{h} \frac{\partial h}{\partial x}\, \frac{\overline{j_x}^2}{\overline{\rho}} \\[2pt]
&\quad
- \frac{\partial}{\partial y}\!\left( \frac{\overline{j_x}\,\overline{j_y}}{\overline{\rho}} \right)
- \frac{1}{h} \frac{\partial h}{\partial y}\, \frac{\overline{j_x}\,\overline{j_y}}{\overline{\rho}} \\[2pt]
&\quad
+ \frac{\partial \overline{\tau_{xx}}}{\partial x}
+ \frac{1}{h} \frac{\partial h}{\partial x}\, \left( \overline{\tau_{xx}} - \tau_{xx}\vert_{z=h_2} \right) \\[2pt]
&\quad
+ \frac{\partial \overline{\tau_{xy}}}{\partial y} \\[2pt]
&\quad
+ \frac{1}{h}\, \tau_{xz}\Big\vert^{z=h_2}_{z=h_1} \\[2pt]
&\quad
- \frac{1}{h} \frac{\partial h}{\partial t}\, \overline{j_x}
\end{aligned}
$$

- **R2Tx**: Time derivative
- **R21x**: Pressure gradient
- **R22xx**, **R22xxS**: Momentum flux $j_x j_x$ + height source
- **R22yx**, **R22yxS**: Momentum flux $j_x j_y$ + height source
- **R23x**, **R23xS**: Normal stress $\tau_{xx}$ + height source (not implemented)
- **R23xy**: In-plane shear stress $\tau_{xy}$ (simplified: $\eta \, \partial u / \partial y$)
- **R24x**: Wall shear stress $\tau_{xz}$
- **R25x**: Height change over time (not implemented)
- **R2Stabx**: Momentum stabilization

## Momentum y

$$
\begin{aligned}
0 &= 
- \frac{\overline{j_y} - \overline{j_y}_{\mathrm{prev}}}{\Delta t} \\[2pt]
&\quad
- \frac{\partial p}{\partial y} \\[2pt]
&\quad
- \frac{\partial}{\partial x}\!\left( \frac{\overline{j_x}\,\overline{j_y}}{\overline{\rho}} \right)
- \frac{1}{h} \frac{\partial h}{\partial x}\, \frac{\overline{j_x}\,\overline{j_y}}{\overline{\rho}} \\[2pt]
&\quad
- \frac{\partial}{\partial y}\!\left( \frac{\overline{j_y}^2}{\overline{\rho}} \right)
- \frac{1}{h} \frac{\partial h}{\partial y}\, \frac{\overline{j_y}^2}{\overline{\rho}} \\[2pt]
&\quad
+ \frac{\partial \overline{\tau_{xy}}}{\partial x} \\[2pt]
&\quad
+ \frac{\partial \overline{\tau_{yy}}}{\partial y}
+ \frac{1}{h} \frac{\partial h}{\partial y}\, \left( \overline{\tau_{yy}} - \tau_{yy}\vert_{z=h_2} \right) \\[2pt]
&\quad
+ \frac{1}{h}\, \tau_{yz}\Big\vert^{z=h_2}_{z=h_1} \\[2pt]
&\quad
- \frac{1}{h} \frac{\partial h}{\partial t}\, \overline{j_y}
\end{aligned}
$$

- **R2Ty**: Time derivative
- **R21y**: Pressure gradient
- **R22xy**, **R22xyS**: Momentum flux $j_x j_y$ + height source
- **R22yy**, **R22yyS**: Momentum flux $j_y j_y$ + height source
- **R23yx**: In-plane shear stress $\tau_{xy}$ (simplified: $\eta \, \partial v / \partial x$)
- **R23y**, **R23yS**: Normal stress $\tau_{yy}$ + height source (not implemented)
- **R24y**: Wall shear stress $\tau_{yz}$
- **R25y**: Height change over time (not implemented)
- **R2Staby**: Momentum stabilization

## Energy

$$
\begin{aligned}
0 &= 
-\,\frac{\overline{E} - \overline{E}_{\text{prev}}}{\Delta t} \\[6pt]
&\quad
-\,\frac{\partial (\overline{E}\, \overline{u})}{\partial x}
- \overline{E}\, \overline{u}\,\frac{1}{h}\,\frac{\partial h}{\partial x} \\[6pt]
&\quad
-\,\frac{\partial (\overline{E}\, \overline{v})}{\partial y}
- \overline{E}\, \overline{v}\,\frac{1}{h}\,\frac{\partial h}{\partial y} \\[6pt]
&\quad
-\,\frac{\partial (p\,\overline{u})}{\partial x}
- p\,\overline{u}\,\frac{1}{h}\,\frac{\partial h}{\partial x} \\[6pt]
&\quad
-\,\frac{\partial (p\,\overline{v})}{\partial y}
- p\,\overline{v}\,\frac{1}{h}\,\frac{\partial h}{\partial y} \\[6pt]
&\quad
+\,\frac{\partial}{\partial x}\!\left(\overline{\tau_{xx}}\,\overline{u}\right)
+ \left(\overline{\tau_{xx}}\,\overline{u}\right)\frac{1}{h}\,\frac{\partial h}{\partial x} \\[6pt]
&\quad
+\,\frac{\partial}{\partial y}\!\left(\overline{\tau_{yy}}\,\overline{v}\right)
+ \left(\overline{\tau_{yy}}\,\overline{v}\right)\frac{1}{h}\,\frac{\partial h}{\partial y} \\[6pt]
&\quad
-\,\frac{1}{h}\left(\tau_{xz,\text{bot}}\,U + \tau_{yz,\text{bot}}\,V\right) \\[6pt]
&\quad
+\,k\,\left( \frac{\partial^2 \overline{T}}{\partial x^2} + \frac{\partial^2 \overline{T}}{\partial y^2} \right) \\[6pt]
&\quad
+ S_{\text{walls}} \\[6pt]
&\quad
- \overline{E}\,\frac{1}{h}\,\frac{\partial h}{\partial t}
\end{aligned}
$$

- **R3T**: Time derivative
- **R31x**, **R31Sx**: Energy advection in x + height source
- **R31y**, **R31Sy**: Energy advection in y + height source
- **R32x**, **R32Sx**: Pressure work in x + height source
- **R32y**, **R32Sy**: Pressure work in y + height source
- **R33x**: Shear work $\tau_{xx}$ in x + height source
- **R33y**: Shear work $\tau_{yy}$ in y + height source
- **R34**: Wall shear work (combines x and y wall velocities)
- **R35x**, **R35y**: Thermal diffusion in x and y
- **R36**: Wall heat balance [see theory](./10_wall_heat.ipynb)
- **R37**: Height change over time